In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from ingredient_parser import ingredient_parser
import pickle
import unidecode, ast

# Top-N recomendations order by score
def get_recommendations(N, scores):
    # load in recipe dataset 
    df_recipes = pd.read_csv('dataset/parsed_data.csv')
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations 
    recommendation = pd.DataFrame(columns = ['score', 'recipe', 'ingredients', 'directions', 'url'])
    count = 0
    for i in top:
        recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i][0][0]))
        recommendation.at[count, 'recipe'] = title_parser(df_recipes['title'][i])
        recommendation.at[count, 'ingredients'] = ingredient_parser_final(df_recipes['ingredients'][i])
        recommendation.at[count, 'directions'] = df_recipes['directions'][i]
        recommendation.at[count, 'url'] = df_recipes['link'][i]
        # recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i]))
        # recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i][0]))
        count += 1
    return recommendation

# neaten the ingredients being outputted 
def ingredient_parser_final(ingredient):
    if isinstance(ingredient, list):
        ingredients = ingredient
    else:
        ingredients = ast.literal_eval(ingredient)
    
    ingredients = ','.join(ingredients)
    ingredients = unidecode.unidecode(ingredients)
    return ingredients

def title_parser(title):
    title = unidecode.unidecode(title)
    return title 

def RecSys(ingredients, N=10):
    """
    The reccomendation system takes in a list of ingredients and returns a list of top 5 
    recipes based of of cosine similarity. 
    :param ingredients: a list of ingredients
    :param N: the number of reccomendations returned 
    :return: top 5 reccomendations for cooking recipes
    """

    # load in tdidf model and encodings 
    with open('tfidf_encodings.pkl', 'rb') as f:
        tfidf_encodings = pickle.load(f)
        
    with open('tfidf.pkl', "rb") as f:
        tfidf = pickle.load(f)

    # parse the ingredients using my ingredient_parser 
    try: 
        ingredients_parsed = ingredient_parser(ingredients)
    except:
        ingredients_parsed = ingredient_parser([ingredients])
    
    # use our pretrained tfidf model to encode our input ingredients
    ingredients_tfidf = tfidf.transform([ingredients_parsed])

    # calculate cosine similarity between actual recipe ingreds and test ingreds
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_encodings)
    scores = list(cos_sim)

    # Filter top N recommendations 
    recommendations = get_recommendations(N, scores)
    return recommendations

In [3]:
if __name__ == "__main__":
    # test ingredients
    test_ingredients = "jalapeno onion beef egg bun lettuce chicken flour brinjal beans onion cheese butter cream pepper"
    recs = RecSys(test_ingredients)
    print(recs)
    # print(recs.score)

   score                           recipe  \
0  0.485              Taco Salad Chip Dip   
1  0.477         Taco-Filled Green Pepper   
2  0.439         Cheeseburger Potato Soup   
3  0.395  Quick Coffee Cake(6 Servings)     
4  0.372          Prize-Winning Meat Loaf   
5  0.366            Potato And Cheese Pie   
6  0.329                      Creamy Corn   
7  0.317                    Chicken Funny   
8  0.316            Jewell Ball'S Chicken   
9  0.310                     Chicken Stew   

                                         ingredients  \
0  8 oz. Ortega taco sauce,8 oz. sour cream,8 oz....   
1  1 lb. Ground Beef,1 pkg. taco seasoning,1 san ...   
2  6 baking potatoes,1 lb. of extra lean ground b...   
3  3 Tbsp. softened butter or margarine,1/2 c. gr...   
4  1 1/2 lb. ground beef,1 c. tomato juice,3/4 c....   
5  3 eggs,1 tsp. salt,1/4 tsp. pepper,2 c. half a...   
6  2 (16 oz.) pkg. frozen corn,1 (8 oz.) pkg. cre...   
7  1 large whole chicken,2 (10 1/2 oz.) cans chic...   
